In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 49.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=29bfda240db2c3b60002177c1b4a8fe15c9e53500d05467603e1a0838c49ac60
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
df3 = spark.read.options(inferSchema='True', delimiter=';') \
  .csv("retail-data-full.csv")

In [8]:
df3.show(19)

+------+------+--------------------+---+-------------------+----+-------+--------------+
|   _c0|   _c1|                 _c2|_c3|                _c4| _c5|    _c6|           _c7|
+------+------+--------------------+---+-------------------+----+-------+--------------+
|536365|85123A|WHITE HANGING HEA...|  6|2010-12-01 08:26:00|2.55|17850.0|United Kingdom|
|536365| 71053| WHITE METAL LANTERN|  6|2010-12-01 08:26:00|3.39|17850.0|United Kingdom|
|536365|84406B|CREAM CUPID HEART...|  8|2010-12-01 08:26:00|2.75|17850.0|United Kingdom|
|536365|84029G|KNITTED UNION FLA...|  6|2010-12-01 08:26:00|3.39|17850.0|United Kingdom|
|536365|84029E|RED WOOLLY HOTTIE...|  6|2010-12-01 08:26:00|3.39|17850.0|United Kingdom|
|536365| 22752|SET 7 BABUSHKA NE...|  2|2010-12-01 08:26:00|7.65|17850.0|United Kingdom|
|536365| 21730|GLASS STAR FROSTE...|  6|2010-12-01 08:26:00|4.25|17850.0|United Kingdom|
|536366| 22633|HAND WARMER UNION...|  6|2010-12-01 08:28:00|1.85|17850.0|United Kingdom|
|536366| 22632|HAND W

In [9]:
df = df3.selectExpr("_c0 as InvoiceNo","_c1 as StockCode","_c2 as Description","_c3 as Quantity", "_c4 as InvoiceData","_c5 as Amount","_c6 as CustomerID","_c7 as Country")

In [10]:
df.show(10)

+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceData|Amount|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|  2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|  2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|  7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|2010

In [22]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [44]:
spark = SparkSession.builder.appName("RetailData").getOrCreate()

schema = StructType() \
      .add("InvoiceNo",StringType(),True) \
      .add("StockCode",StringType(),True) \
      .add("Description",StringType(),True) \
      .add("Quantity",IntegerType(), True) \
      .add("Invoicedata", StringType(), True)\
      .add("Amount",StringType(),True)\
      .add("CustomerID",StringType(),True)\
      .add("Country",StringType(),True)

df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(schema) \
      .load("retail-data-full.csv" ,delimiter = ';')
df_with_schema.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Invoicedata: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [45]:
df_with_schema.show(10)

+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        Invoicedata|Amount|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|  2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|  7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|2010-12-01 08:26:00|  4.25|   17850.0|United Kingdom|
|   536366|    22633|HAND WARMER UNION...|       6|2010

In [68]:
df_with_schema.select("InvoiceData").show(10, truncate = False)

+-------------------+
|InvoiceData        |
+-------------------+
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:28:00|
|2010-12-01 08:28:00|
|2010-12-01 08:34:00|
|2010-12-01 08:34:00|
+-------------------+
only showing top 10 rows



In [69]:
results = df_with_schema

In [71]:
print(results)
results.show(10)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, Invoicedata: string, Amount: string, CustomerID: string, Country: string]
+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        Invoicedata|Amount|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+------+----------+--------------+
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|  2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|  3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|  7.65|   17850.0|United Kingdom|
| 